In [1]:
# Importação das bibliotecas

import cx_Oracle
import pandas as pd
import unicodedata

In [2]:
# Conexão com o banco de dados Oracle, execução da consulta e leitura dos dados

uid = "********"  # usuário
pwd = "********"  # senha
db = "*********"  # string de conexão do Oracle, configurado no
# cliente Oracle, arquivo tnsnames.ora

connection = cx_Oracle.connect(uid + "/" + pwd + "@" + db)  # cria a conexão

query = """select ******* Data_Classificacao, ******* Codigo_Produto, ******* Descricao_Produto,
******* Derivacao_Produto, ******* Codigo_Fornecedor, ******** Nome_Fornecedor, sum(*******) Quantidade_KG 
from ******* a, ******* b, ******* c where ******* between to_date('01/01/2010', 'dd/mm/yyyy') and
to_date('31/12/2021', 'dd/mm/yyyy') and ******* in (1, 2, 3, 4, 5, 9673, 1325) and ******* = ******* and
******* = ******* and ******* = 1 group by *******, *******, ********, ********, *********,
******** order by **********""" # Consulta que será executada

df = pd.read_sql(query, con = connection) # Execução e leitura dos dados retornados pela consulta

In [ ]:
df.head() # Exibição dos dados

In [ ]:
# Geração de uma nova coluna que indica a que safra o registro pertence

for index, row in df.iterrows():
    df.loc[index, 'SAFRA'] = str(df.loc[index, 'DATA_CLASSIFICACAO'].year)   
    
df.head() # Exibição dos dados    

In [ ]:
# Geração de uma nova coluna que indica o calibre da fruta, extraído da coluna "DERIVACAO_PRODUTO"

for index, row in df.iterrows():
    if (df.loc[index, 'DERIVACAO_PRODUTO'] != 'GRA/BIN') and (df.loc[index, 'DERIVACAO_PRODUTO'] != 'INDP') and (df.loc[index, 'DERIVACAO_PRODUTO'] != 'INDPP') and (df.loc[index, 'DERIVACAO_PRODUTO'] != 'IND'):
        peso_bin, calibre = df.loc[index, 'DERIVACAO_PRODUTO'].split("/")
        df.loc[index, 'CALIBRE'] = calibre
    else:    
        df.loc[index, 'CALIBRE'] = df.loc[index, 'DERIVACAO_PRODUTO']
        
df.head() # Exibição dos dados

In [ ]:
# Geração de uma nova coluna que indica o agrupamento de tamanho da fruta, extraído da coluna "CALIBRE"

for index, row in df.iterrows():
    if ((df.loc[index, 'CALIBRE'] >= '060') and (df.loc[index, 'CALIBRE'] <= '120')) or (df.loc[index, 'CALIBRE'] == 'G'):
        agp_tamanho = 'G'
        df.loc[index, 'AGRUPAMENTO_TAMANHO'] = agp_tamanho
    elif ((df.loc[index, 'CALIBRE'] >= '135') and (df.loc[index, 'CALIBRE'] <= '165')) or (df.loc[index, 'CALIBRE'] == 'M'):   
        agp_tamanho = 'M'
        df.loc[index, 'AGRUPAMENTO_TAMANHO'] = agp_tamanho
    elif ((df.loc[index, 'CALIBRE'] >= '180') and (df.loc[index, 'CALIBRE'] <= '300')) or (df.loc[index, 'CALIBRE'] == 'P'):
        agp_tamanho = 'P'
        df.loc[index, 'AGRUPAMENTO_TAMANHO'] = agp_tamanho
    elif (df.loc[index, 'CALIBRE'] == 'DVR') or (df.loc[index, 'CALIBRE'] == 'GRA/BIN'):
        agp_tamanho = 'OUTROS'
        df.loc[index, 'AGRUPAMENTO_TAMANHO'] = agp_tamanho
    elif (df.loc[index, 'CALIBRE'] == 'IND') or (df.loc[index, 'CALIBRE'] == 'INDP') or (df.loc[index, 'CALIBRE'] == 'INDPP'):
        agp_tamanho = 'IND'
        df.loc[index, 'AGRUPAMENTO_TAMANHO'] = agp_tamanho    
        
df.head() # Exibição dos dados

In [ ]:
# Substituição das informações da coluna NOME_FORNECEDOR de acordo com o nome do pomar a que pertence o registro

df.loc[df.CODIGO_FORNECEDOR == 1, 'NOME_FORNECEDOR'] = 'POMAR GERAL'
df.loc[df.CODIGO_FORNECEDOR == 2, 'NOME_FORNECEDOR'] = 'POMAR 1'
df.loc[df.CODIGO_FORNECEDOR == 3, 'NOME_FORNECEDOR'] = 'POMAR 6'
df.loc[df.CODIGO_FORNECEDOR == 4, 'NOME_FORNECEDOR'] = 'POMAR 3'
df.loc[df.CODIGO_FORNECEDOR == 5, 'NOME_FORNECEDOR'] = 'POMAR 2'
df.loc[df.CODIGO_FORNECEDOR == 1325, 'NOME_FORNECEDOR'] = 'POMAR 4'
df.loc[df.CODIGO_FORNECEDOR == 9673, 'NOME_FORNECEDOR'] = 'POMAR 5'

df.head() # Exibição dos dados 

In [ ]:
# Remoção de caracteres especiais e acentos da descrição dos produtos

for index, row in df.iterrows():
    processamento = unicodedata.normalize("NFD", df.loc[index, 'DESCRICAO_PRODUTO'])
    processamento = processamento.encode("ascii", "ignore")
    processamento = processamento.decode("utf-8")
    df.loc[index, 'DESCRICAO_PRODUTO'] = processamento
    
df.head() # Exibição dos dados

In [10]:
df.info() # Visualização do tipo de dados de cada coluna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91242 entries, 0 to 91241
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATA_CLASSIFICACAO   91242 non-null  datetime64[ns]
 1   CODIGO_PRODUTO       91242 non-null  object        
 2   DESCRICAO_PRODUTO    91242 non-null  object        
 3   DERIVACAO_PRODUTO    91242 non-null  object        
 4   CODIGO_FORNECEDOR    91242 non-null  int64         
 5   NOME_FORNECEDOR      91242 non-null  object        
 6   QUANTIDADE_KG        91242 non-null  float64       
 7   SAFRA                91242 non-null  object        
 8   CALIBRE              91242 non-null  object        
 9   AGRUPAMENTO_TAMANHO  91242 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 4.5+ MB


In [11]:
# Exportação dos dados transformados para um arquivo .CSV

df.to_csv('C:/Users/*********/Documents/Projetos Jupyter Notebook/TCC - Ciência de Dados/Dados_Produção.csv', sep = ';')